# NFT image and traits generator

This scripts will collect layers from `image-sources` folder and build images for your NFT. It also will put this traits into JSON files for easy future deploy.

Make sure you have edited this script as you need.

In [22]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [23]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%
AllTraits = [
    {
        "name": "Hair",
        "traits":  ["hair1", "hair2", "hair3", "hair4"],
        "weights": [30, 20, 10, 40],
        "folder": "hair"
    },
    {
        "name": "Accessory",
        "traits":  ["accessory1", "accessory2", "accessory3", "accessory4"] ,
        "weights": [25, 25, 25, 25],
        "folder": "accessory"
    },
    {
        "name": "Face",
        "traits":  ["face1", "face2", "face3", "face4"],
        "weights": [25, 25, 25, 25],
        "folder": "face"
    },
    {
        "name": "Body",
        "traits":  ["Duck"],
        "weights": [100],
        "files": ["Duck"],
        "folder": "body"
    },
    {
        "name": "Background",
        "traits":  ["Green","Yellow","Blue","White"],
        "weights": [30, 10, 20, 40],
        "folder": "background"
    }
]

In [24]:
## Generate Traits

TOTAL_IMAGES = 32 # Number of random unique images we want to generate

all_images = [] 

# do check weights
for trait in AllTraits:
    print(trait["name"] +": t: "+ str(len(trait["traits"]))+ ", w: " + str(len(trait["weights"])))

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    for trait in AllTraits:
        new_image[trait["name"]] = random.choices(trait["traits"], trait["weights"])[0]


    # Here you can add custom blacklist and do not generate images with specific traits
    if new_image["Face"] == "face1" and new_image["Accessory"] == "accessory1":
        return create_new_image()
    

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


Hair: t: 4, w: 4
Accessory: t: 4, w: 4
Face: t: 4, w: 4
Body: t: 1, w: 1
Background: t: 4, w: 4


In [25]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [26]:
#Print counts of traits

counts = {}

for trait in AllTraits:
    counts[trait["name"]] = {}
    for item in trait["traits"]:
        counts[trait["name"]][item] = 0

for image in all_images:
    for k,c in counts.items():
        counts[k][image[k]] += 1

print("=====")

for k in counts:
    print(k + ": ")
    print(counts[k])

=====
Hair: 
{'hair1': 5, 'hair2': 8, 'hair3': 1, 'hair4': 18}
Accessory: 
{'accessory1': 5, 'accessory2': 12, 'accessory3': 7, 'accessory4': 8}
Face: 
{'face1': 7, 'face2': 10, 'face3': 6, 'face4': 9}
Body: 
{'Duck': 32}
Background: 
{'Green': 11, 'Yellow': 2, 'Blue': 6, 'White': 13}


In [27]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [28]:
#print(all_images)

In [29]:
# Get Trait Counts

counts = {}

for trait in AllTraits:
    counts[trait["name"]] = {}
    for item in trait["traits"]:
        counts[trait["name"]][item] = 0

for image in all_images:
    for k,c in counts.items():
        counts[k][image[k]] += 1
    
print(counts)

{'Hair': {'hair1': 5, 'hair2': 8, 'hair3': 1, 'hair4': 18}, 'Accessory': {'accessory1': 5, 'accessory2': 12, 'accessory3': 7, 'accessory4': 8}, 'Face': {'face1': 7, 'face2': 10, 'face3': 6, 'face4': 9}, 'Body': {'Duck': 32}, 'Background': {'Green': 11, 'Yellow': 2, 'Blue': 6, 'White': 13}}


In [30]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [31]:
# load an array of all images
trait_img = {}
for trait in AllTraits:
    trait_img[trait["name"]] = {}
    for item in trait["traits"]:
        trait_img[trait["name"]][item] = Image.open(f'./image-source/{trait["folder"]}/{item}.png').convert('RGBA')
        
#watermark = Image.open(f'./watermark.png').convert('RGBA')

#### Generate Images    
for item in all_images:

    im = Image.new('RGBA', (32,32))
    
    for trait in reversed(AllTraits):
        #this_layer = Image.open(f'./image-source/{trait["folder"]}/{item [ trait["name"] ] }.png').convert('RGBA')
        this_layer = trait_img[trait["name"]][item [ trait["name"] ]]
        im = Image.alpha_composite(im, this_layer)


    # WATERMARK
    #im = Image.alpha_composite(im, watermark)

    #Resize
    im = im.resize((128,128), Image.Resampling.NEAREST)

    #Convert to RGB (jpg much faster when we making tons of images. 3h to write 10k 1500x1500 png images VS 10 min to wrtie 10k jpg images)
    im = im.convert('RGB')
    file_name = str(item["tokenId"]) + ".jpg"
    im.save("./images-output/" + file_name, quality=90)
    
    

In [32]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)

# Set your own
IMAGES_BASE_URI = "http:/localhost:8080/token/"
PROJECT_NAME = "EverDucks"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.jpg',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }

    for trait in AllTraits:
        token["attributes"].append(getAttribute(trait["name"], i[ trait["name"] ]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()